# Analyze Product Sentiment

In [1]:
import turicreate

# Read product review data

In [2]:
products = turicreate.SFrame('amazon_baby.sframe')

# Building a sentiment classifier

## Build word count vectors

In [20]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [21]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [59]:
def conteo(dict1,str1):
    if str1 in dict1:
        return dict1[str1]
    else:
        return 0

In [60]:
for word in selected_words:
    products[word] = products['word_count'].apply(lambda x:conteo(x,word))

   # Creo un diccionario con los valores sumados

In [30]:
myset = {(word,products[word].sum()) for word in selected_words}


In [31]:
mysum = {}
for word in selected_words:
    mysum[word] = products[word].sum()



# Answer 1

In [32]:
import pandas as pd
df = pd.DataFrame.from_dict(mysum,'index')
df.columns = ['word']
df = df.sort_values(by='word')
df
#dfsum =turicreate.SFrame(mysum)

,word
wow,461.0
awful,753.0
horrible,1245.0
terrible,1282.0
hate,1285.0
fantastic,1765.0
amazing,2726.0
awesome,4075.0
bad,4950.0
love,43867.0


# Train our sentiment classifier

In [55]:
#positive sentiment = 4-star or 5-star reviews
products = products[products['rating']!= 3]
products['sentiment'] = products['rating'] >= 4

In [34]:
train_data,test_data = products.random_split(.8,seed=0)

In [57]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)


Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 2.434509     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 2.796721     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 3.335362     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 5.248693     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Creo el modelo de answer2

In [61]:
train_data,test_data = products.random_split(.8,seed=0)
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=selected_words, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.425770     | 0.847708          | 0.846385            |

| 5         | 6        | 0.533890     | 0.847708          | 0.846385            |

| 6         | 7        | 0.632357     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

# Answer 2

In [40]:
selected_words_model.coefficients

name,index,class,value,stderr
(intercept),None,1,0.8730523140220043,0.0074209749451630514
awesome,None,1,1.0056865099491044,0.06312207550960001
great,None,1,0.673788800558403,0.013878642610622605
fantastic,None,1,0.778208015543446,0.08618412069305045
amazing,None,1,1.0587756629335572,0.0787371389067602
love,None,1,1.1242227839404169,0.01984496678027315
horrible,None,1,-1.9717454039217406,0.07642609142520146
bad,None,1,-0.9566124758067379,0.033643249260543254
terrible,None,1,-1.9277414656234315,0.07391732820972693
awful,None,1,-1.7280068463019074,0.09513832961728019


# Answer 3 

In [41]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.8571584401418053,
 'auc': 0.8764610783273566,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  5851 |
 |      0       |        1        |  2814 |
 |      1       |        1        | 25581 |
 |      1       |        0        |  2424 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.907127659574468,
 'log_loss': 0.6090684614661309,
 'precision': 0.9008980454305335,
 'recall': 0.9134440278521693,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0

In [62]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8463848186404036,
 'auc': 0.6936022046674926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.39622654670876317,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 53

In [45]:
test_data.num_rows()



36670

In [46]:
for num1 in {1,2,3,4,5}:
    print(test_data[test_data['rating'] == num1].num_rows())

2928
2274
3463
6716
21289


In [49]:
majority = test_data[test_data['rating'] == 5].num_rows()/test_data.num_rows()
majority

0.5805563130624488

# Answer 4

In [73]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')
products['predicted_sentiment_selected'] =  selected_words_model.predict(products, output_type = 'probability')

In [74]:
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']

In [77]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)
diaper_champ_reviews[0]


{'name': 'Baby Trend Diaper Champ',
 'review': "I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out.",
 'rating': 4.0,
 'word_count': {'key': 1.0,
  'have': 1.0,
  'pieces': 1.0,
  'between': 1.0,
  'cushioned': 1.0,
  'piece': 1.0,
  'take': 1.0,
  'overloading': 1.0,
  'be': 1.0,
  'looking': 1.0,
  're': 1.0,
  'stinkiness': 1.0,
  'retain': 1.0,
  'now': 1.0,
  'wonderful': 1.0,
  'worked': 1.0,
  '16': 1.0,
  'and': 3.0,
  'months': 1.0,
  've': 1.0,
  'in': 1.0,
  'us': 1.0,
  'i': 3.0,
  'experienced': 1.0,
  '

In [76]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment_selected', ascending=False)
diaper_champ_reviews

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'sorry': 1.0, 'be': 1.0,'will': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'second': 2.0, 'have':1.0, 'are': 1.0, 'pull': ...",1,0.0
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'parents': 1.0, 'new':1.0, 'recommended': 1.0, ...",1,0.0
Baby Trend Diaper Champ,"Let me just say, I LOVETHIS PRODUCT!! I used ...",5.0,"{'friends': 1.0, 'feces':1.0, 'tank': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'easy': 1.0, 'how': 1.0,'like': 1.0, 'on': 1.0, ...",1,0.0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'plastic': 1.0, 'in':1.0, 'each': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I have been using thisdiaper pail for 41/2 ...,5.0,"{'recommend': 1.0,'would': 1.0, 'with': ...",1,0.0
Baby Trend Diaper Champ,I love this diaper pail.It keeps the diapers ...,4.0,"{'wish': 1.0, 'is': 1.0,'stars': 1.0, 'gave': ...",1,0.0
Baby Trend Diaper Champ,I've worked with kidsmore than half my life. ...,5.0,"{'diapers': 1.0,'soiled': 1.0, 'if': ...",1,0.0
Baby Trend Diaper Champ,I love this diaper pail!It's so easy to use a ...,5.0,"{'some': 1.0, 'save':1.0, 'refills': 1.0, ...",1,0.0


## Show the most positive reviews

In [25]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

# Most negative reivews

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [51]:
giraffe_reviews[-2]['review']

NameError: name 'giraffe_reviews' is not defined